In [ ]:
import pickle

# import packages

import numpy as np
from numpy import random
from scipy import stats
import pandas as pd
import copy
import matplotlib.pyplot as plt

# import r packages if needed

import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
rpackages.importr("clusterGeneration")
rpackages.importr("mpower")
rpackages.importr("base")
rpackages.importr("utils")
cluster_generation = robjects.packages.importr("clusterGeneration")
mpower = robjects.packages.importr("mpower")
base = robjects.packages.importr("base")
utils = robjects.packages.importr("utils")

# set seed from R?
r = robjects.r
set_seed = r('set.seed')
from numpy import load


In [ ]:
# seeds for reproducibility --> NICE :-D
set_seed(8953)
random.seed(8953)

In [ ]:
# SIMULATION SETUP: "META-PARAMETERS"

sample_sizes = [1000,2000,5000,10000]
test_sizes = [500,1000] # TODO: DECIDE
runs = 30 # TODO: DECIDE HOW MANY RUNS (I.E. HOW MANY TIMES GENERATING ALL SETUPS AND RUNNING ALL META LEARNERS ON THEM)
n_setups = 28

In [ ]:
 # Fully Synthetic Data Sets (Set up from Künzel et al.)

# 1: Simulate the d-dimensional X.
# 2: Create Potential Outcomes Y(1) and Y(0).
# 3: Simulate Treatment Assignments trough W.

In [ ]:
########################################################

In [ ]:
# 1: Simulate the d-dimensional X
## TODO: PROBABLY HAVE TO DO THIS ON R: SINCE NO SET.SEED() POSSIBLE!

# Setup parameters
d = 25 # TODO: set dimension according to setup
N = 1000 # TODO: set sample size according to setup

# X Correlation matrix and mean
mean = np.zeros(d) # TODO: set mean according to setup
cov = np.array(mpower.cvine(d=d, alpha = 0.5, beta = 0.5)) # TODO: set cov according to setup

# Simulate X
X = random.multivariate_normal(mean=mean, cov=cov, size=N, check_valid='warn')

In [ ]:
X

In [ ]:
# This will be all the "datasets", i.e. all simulation setups * simulation runs
data = []
for i in range(n_setups):
    data.append([])

In [ ]:
## EXAMPLE!

# make 10 runs of dataset X
# whole data generation procedure in this loop FOR EACH SETUP!!



d = 25 # TODO: set dimension according to setup
N = 1000 # TODO: set sample size according to setup
mean = np.zeros(d) # TODO: set mean according to setup

for i in range(runs):

    # 1 GENERATE FEATURES X
    cov = np.array(mpower.cvine(d=d, alpha = 0.5, beta = 0.5)) # TODO: set cov according to setup
    X = random.multivariate_normal(mean=mean, cov=cov, size=N, check_valid='warn')

    # 2 GENERATE ERRORS E_0 & E_1
    e_0 = random.normal(loc=0.0,scale=1.0,size=N)
    e_1 = random.normal(loc=0.0,scale=1.0,size=N)

    # 3 COMPUTE MU_0 & MU_1 (SIMPLE CATE, LINEAR, NO CONFOUNDING)
    betas_0 = random.uniform(low=-5, high=5, size=d)
    mu_0 = np.matmul(X,betas_0) + 5*X[:,0]
    mu_1 = mu_0 + 8*X[:,1] # linear
    tau = mu_1 - mu_0

    # 4 CREATE POTENTIAL OUTCOMES Y_0 & Y_1
    Y_0 = mu_0 + e_0
    Y_1 = mu_1 + e_1

    # 5 SET PROPENSITY SCORE E_X (CONSTANT, BALANCED)
    e_x = 0.5

    # 6 GENERATE TREATMENT ASSIGMENT W
    W = random.binomial(size=N, n=1, p=e_x)

    # 7 CREATE OBSERVED VARIABLES Y
    Y = np.multiply(W,Y_1) + np.multiply(np.ones(N)-W,Y_0)

    # 8 CREATE DATASET
    dataset = np.concatenate((np.reshape(Y,(N,1)),X,np.reshape(W,(N,1)),np.reshape(tau,(N,1))), axis = 1)

    # 9 APPEND RUN TO LIST
    data[0].append(dataset)

In [ ]:
data[0][0]

In [ ]:
print(len(data)) # setups
print(len(data[0])) # runs
print(len(data[0][0])) # rows
print(len(data[0][0][0])) # columns


In [ ]:
# Check X
X

In [ ]:
# Check cov
cov

In [ ]:
#######################################################

In [ ]:
# 2: Create Potential Outcomes Y(1) and Y(0).

# 2.1 Simulate errors, FIX
e_0 = random.normal(loc=0.0,scale=1.0,size=N)
e_1 = random.normal(loc=0.0,scale=1.0,size=N)

# if needed take student_t distributed errors to see simulation with heavy-tailed errors

# e_o = random.standard_t(df=1,size=N)
# e_1 = random.standard_t(df=1,size=N)

In [ ]:
#######################################################

In [ ]:
# 2.2 Create Response Functions

In [ ]:
# SI1.1 simple cate - indicator  (no confounding)
betas_0 = random.uniform(low=-5, high=5, size=d)
mu_0 = np.matmul(X,betas_0) + 5*np.int8(X[:,0]>0.5)
mu_1 = mu_0 + 8*np.int8(X[:,1]>0.1) # indicator
tau = mu_1 - mu_0
tau

In [ ]:
# SI1.2 simple cate - linear (no confounding)
betas_0 = random.uniform(low=-5, high=5, size=d)
mu_0 = np.matmul(X,betas_0) + 5*X[:,0]
mu_1 = mu_0 + 8*X[:,1] # linear
tau = mu_1 - mu_0
tau

In [ ]:
# SI1.3 simple cate - quadratic (no confounding)
betas_0 = random.uniform(low=-5, high=5, size=d)
mu_0 = np.matmul(X,betas_0) + 5*(X[:,0]**2)
mu_1 = mu_0 + 8*(X[:,1]**2) # quadratic
tau = mu_1 - mu_0
tau

In [ ]:
#############################################

In [ ]:
# SI2 complex linear cate  (in Künzel it is low=1, high=30)
betas_0 = random.uniform(low=-15, high=15, size=d)
betas_1 = random.uniform(low=-15, high=15, size=d)
mu_0 = np.matmul(X,betas_0)
mu_1 = np.matmul(X,betas_1)
tau = mu_1 - mu_0

In [ ]:
###############################################

In [ ]:
def varsigma_funct(x):
    return 2/(1+np.exp(-12*(x-1/2)))

In [ ]:
# SI3 complex non-linear
mu_0 = -1/2*varsigma_funct(x=X[:,0])*varsigma_funct(x=X[:,1])
mu_1 = 1/2*varsigma_funct(x=X[:,0])*varsigma_funct(x=X[:,1])
tau = mu_1 - mu_0

In [ ]:
#############################################

In [ ]:
# SI4 no treatment effect (global linear response)
betas_noT = random.uniform(low=-15, high=15, size=d)
mu_0 = np.matmul(X,betas_noT)
mu_1 = mu_0
tau = np.zeros(N)

In [ ]:
###########################################

In [ ]:
# SI5 no treatment effect (piecewise linear response)
betas = np.random.uniform(low=-15, high=15, size=d)

betas_l = copy.copy(betas) # betas_lower
betas_l[5:d] = 0

betas_m = copy.copy(betas) #betas_middle
betas_m[0:5] = 0
betas_m[10:d] = 0

betas_u = copy.copy(betas) #betas_upper
betas_u[0:10] = 0
betas_u[15:d] = 0

def piecewise_linear_new(x):
    condition_l = x[:, 19] < -0.4
    condition_u = x[:, 19] > 0.4

    array = np.zeros(N)
    array[condition_l] = np.matmul(x[condition_l, :], betas_l)
    array[~condition_l & ~condition_u] = np.matmul(x[~condition_l & ~condition_u, :], betas_m)
    array[condition_u] = np.matmul(x[condition_u, :], betas_u)

    return array

mu_0 = piecewise_linear_new(X)

mu_1 = mu_0

tau = np.zeros(N)

# TODO: CHECK IF RIGHT

In [ ]:
# SI5 no treatment effect (piecewise linear response)
betas = np.random.uniform(low=-15, high=15, size=d)

betas_l = copy.copy(betas) # betas_lower
betas_l[5:d] = 0

betas_m = copy.copy(betas) #betas_middle
betas_m[0:5] = 0
betas_m[10:d] = 0

betas_u = copy.copy(betas) #betas_upper
betas_u[0:10] = 0
betas_u[15:d] = 0

In [ ]:
def piecewise_linear_new(x):
    condition_l = x[:, 19] < -0.4
    condition_u = x[:, 19] > 0.4

    array = np.zeros(N)
    array[condition_l] = np.matmul(x[condition_l, :], betas_l)
    array[~condition_l & ~condition_u] = np.matmul(x[~condition_l & ~condition_u, :], betas_m)
    array[condition_u] = np.matmul(x[condition_u, :], betas_u)

    return array

In [ ]:
##########################################

In [ ]:
# SI6.1 beta confounded, no treatment effect
X = random.uniform(low=0, high=1, size=(N,d)) # ACHTUNG: nöd wiederhole!
mu_0 = 2*X[:,0] - 1
mu_1 = mu_0
tau = np.zeros(N)


In [ ]:
# SI6.2 beta confounded, simple cate (indicator)
X = random.uniform(low=0, high=1, size=(N,d))
mu_0 = 2*X[:,0] - 1
mu_1 = mu_0 + 2*np.int8(X[:,1]>0.4)
tau = mu_1 - mu_0
tau

In [ ]:
# SI6.3 beta confounded, simple cate (linear)
X = random.uniform(low=0, high=1, size=(N,d))
mu_0 = 2*X[:,0] - 1
mu_1 = mu_0 + 2*X[:,1]
tau = mu_1 - mu_0


In [ ]:
# SI6.4 beta confounded, simple cate (quadratic)
X = random.uniform(low=0, high=1, size=(N,d))
mu_0 = 2*X[:,0] - 1
mu_1 = mu_0 + 2*(X[:,1]**2)
tau = mu_1 - mu_0
tau

In [ ]:
##################################

In [ ]:
# SI7.1 beta confounded, complex cate (linear)
betas = random.uniform(low=-15, high=15, size=d)

mu_0 = 2*X[:,0] - 1
mu_1 = mu_0 + np.matmul(X,betas)
tau = mu_1 - mu_0

# TODO: CHECK WHETHER THIS SETUP MAKES SENSE

In [ ]:
# SI7.2 beta confounded, complex cate (non-linear)
mu_0 = 2*X[:,0] - 1
mu_1 = mu_0 + 1/2*varsigma_funct(x=X[:,0])*varsigma_funct(x=X[:,1])

# TODO: CHECK WHETER THIS SETUP MAKES SENSE

In [ ]:
# TODO: check relationship between mu_0 & mu_1 and see whether this makes sense
check_values_0 = 2*X[:,0] - 1
check_values_1 = mu_0 + 1/2*varsigma_funct(x=X[:,0])*varsigma_funct(x=X[:,1])
check_values_2 = mu_0 + np.matmul(X,betas)

plt.plot(check_values_0, check_values_1, 'bo')
plt.show()

In [ ]:
###################################
# 2.3 Creat Potential Outcomes, FIX
Y_0 = mu_0 + e_0
Y_1 = mu_1 + e_1

In [ ]:
# quick check Y_0
Y_0

In [ ]:
# Quick Check Y_1
Y_1

In [ ]:
###################################
# 3.1 Propensity score setups # TODO: change for setup

# i) constant, balanced
e_x = 0.5

# ii) constant, unbalanced
e_x = 0.01

In [ ]:
#################################

In [ ]:
# iii) beta confounded (balanced)
beta_dist = stats.beta(a=2, b=4) # set beta distribution
cdf_values = stats.norm.cdf(X[:,0])
beta_values = beta_dist.pdf(cdf_values) # calculate pdf values for x1
e_x = 1/4*(1+beta_values)

pd.DataFrame(e_x).describe() # summary stats of e_x

In [ ]:
pd.DataFrame(e_x).hist(bins=20)

In [ ]:
# check if balanced
W = random.binomial(size=N, n=1, p=e_x)
pd.DataFrame(W).describe()

In [ ]:
#####################################

In [ ]:
# iv) beta confounded (unbalanced)
beta_dist = stats.beta(a=2, b=4) # set beta distribution
cdf_values = stats.norm.cdf(X[:,0])
beta_values = beta_dist.pdf(cdf_values) # calculate pdf values for x1
e_x = 1/20*(1+beta_values)

pd.DataFrame(e_x).describe() # summary stats of e_x

In [ ]:
pd.DataFrame(e_x).hist(bins=20) # summary stats of e_x

In [ ]:
# check if unbalanced and how much
W = random.binomial(size=N, n=1, p=e_x)
pd.DataFrame(W).describe()

In [ ]:
beta_dist = stats.beta(a=2, b=4) # set beta distribution
cdf_values = stats.norm.cdf(X[:,0])
beta_values = beta_dist.pdf(cdf_values) # calculate pdf values for x1
e_x = 1/4*(1+beta_values)

pd.DataFrame(e_x).describe() # summary stats of e_x

In [ ]:
pd.DataFrame(e_x).hist(bins=20) # summary stats of e_x

In [ ]:
W = random.binomial(size=N, n=1, p=e_x)
pd.DataFrame(W).describe()

In [ ]:
##########################################

In [ ]:
X

In [ ]:
###################################

In [ ]:
# 3.2 Simulate Treatment Assignments trough W

# Simulate Treatment Assignment, FIX
W = random.binomial(size=N, n=1, p=e_x)

# Create Observed Outcome, FIX
ones = np.ones(N)
Y = np.multiply(W,Y_1) + np.multiply(ones-W,Y_0)

In [ ]:
pd.DataFrame(e_x).describe() # summary stats of e_x

In [ ]:
# check out observed outcomes
Y

In [ ]:
pd.DataFrame(Y).describe() # summary stats of Y

In [ ]:
X

In [ ]:
Y

In [ ]:
W

In [ ]:
tau

In [ ]:
dataset = np.concatenate((np.reshape(Y,(N,1)),X,np.reshape(W,(N,1)),np.reshape(tau,(N,1))), axis = 1)
dataset

In [ ]:
data.apppend(dataset)

# Helper function to generate datasets

In [ ]:
exs = []
cates = []
sample_sizes = []
test_sizes = []
runs = 30

In [ ]:
# all settings of e_x
exs = [0.5,0.5,0.5,0.5,0.5,0.5,0.5,
       0.1,0.1,0.1,0.1,0.1,0.1,0.1,
       'beta_bal','beta_bal','beta_bal','beta_bal','beta_bal','beta_bal','beta_bal',
       'beta_unbal','beta_unbal','beta_unbal','beta_unbal','beta_unbal','beta_unbal','beta_unbal']

In [ ]:
# all settings of mu_0/mu_1
cates = ['glinear','plinear','sindcate','slinearcate','squadcate','clinearcate','cnonlincate']*4
cates

In [ ]:
def generate(ex, cate, sample_size): # TODO: add test_size

    # 1 GENERATE FEATURES X
    cov = np.array(mpower.cvine(d=d, alpha = 0.5, beta = 0.5)) # TODO: set cov according to setup
    X = random.multivariate_normal(mean=mean, cov=cov, size=sample_size, check_valid='warn')

    # 2 GENERATE ERRORS E_0 & E_1
    e_0 = random.normal(loc=0.0,scale=1.0,size=sample_size)
    e_1 = random.normal(loc=0.0,scale=1.0,size=sample_size)

    # 3 COMPUTE MU_0 & MU_1 --> based on setting
    if cate == 'glinear':
        betas_noT = random.uniform(low=-5, high=5, size=d)
        mu_0 = np.matmul(X,betas_noT)
        mu_1 = mu_0
        tau = np.zeros(sample_size)

    elif cate == 'plinear':
        betas = np.random.uniform(low=-5, high=5, size=d)
        betas_l = copy.copy(betas) # betas_lower
        betas_l[5:d] = 0
        betas_m = copy.copy(betas) #betas_middle
        betas_m[0:5] = 0
        betas_m[10:d] = 0
        betas_u = copy.copy(betas) #betas_upper
        betas_u[0:10] = 0
        betas_u[15:d] = 0

        mu_0 = piecewise_linear_new(X)
        mu_1 = mu_0
        tau = np.zeros(sample_size)


    elif cate == 'sindcate':
        betas_0 = random.uniform(low=-5, high=5, size=d)
        mu_0 = np.matmul(X,betas_0) + 5*np.int8(X[:,0]>0.5)
        mu_1 = mu_0 + 8*np.int8(X[:,1]>0.1) # indicator
        tau = mu_1 - mu_0

    elif cate == 'slinearcate':
        betas_0 = random.uniform(low=-5, high=5, size=d)
        mu_0 = np.matmul(X,betas_0) + 5*X[:,0]
        mu_1 = mu_0 + 8*X[:,1] # linear
        tau = mu_1 - mu_0

    elif cate == 'squadcate':
        betas_0 = random.uniform(low=-5, high=5, size=d)
        mu_0 = 2*X[:,0] - 1
        mu_1 = mu_0 + 2*(X[:,1]**2)
        tau = mu_1 - mu_0

    elif cate == 'clinearcate':
        betas_0 = random.uniform(low=-5, high=5, size=d)
        betas_1 = random.uniform(low=-5, high=5, size=d)
        mu_0 = np.matmul(X,betas_0)
        mu_1 = np.matmul(X,betas_1)
        tau = mu_1 - mu_0

    elif cate == 'cnonlincate':
        mu_0 = 2*X[:,0] - 1
        mu_1 = mu_0 + 1/2*varsigma_funct(x=X[:,0])*varsigma_funct(x=X[:,1])
        tau = mu_1 - mu_0

    else:
        raise(NotImplementedError)





    # 4 CREATE POTENTIAL OUTCOMES Y_0 & Y_1
    Y_0 = mu_0 + e_0
    Y_1 = mu_1 + e_1

    # 5 SET PROPENSITY SCORE E_X --> based on setting
    if isinstance(ex,float):
        e_x = ex

    elif ex == 'beta_balanced':
        beta_dist = stats.beta(a=2, b=4) # set beta distribution
        cdf_values = stats.norm.cdf(X[:,0])
        beta_values = beta_dist.pdf(cdf_values) # calculate pdf values for x1
        e_x = 1/4*(1+beta_values)

    elif ex == 'beta_unbalanced':
        beta_dist = stats.beta(a=2, b=4) # set beta distribution
        cdf_values = stats.norm.cdf(X[:,0])
        beta_values = beta_dist.pdf(cdf_values) # calculate pdf values for x1
        e_x = 1/20*(1+beta_values)


    # 6 GENERATE TREATMENT ASSIGMENT W
    W = random.binomial(size=sample_size, n=1, p=e_x)

    # 7 CREATE OBSERVED VARIABLES Y
    Y = np.multiply(W,Y_1) + np.multiply(np.ones(sample_size)-W,Y_0)

    # 8 CREATE DATASET
    dataset = np.concatenate((np.reshape(Y,(sample_size,1)),X,np.reshape(W,(sample_size,1)),np.reshape(tau,(sample_size,1))), axis = 1)

    # 9 APPEND RUN TO LIST
    return dataset





In [ ]:
# def generate(): --> für ein dataset!

# loop over setups (28)
## loop over sample_sizes (4)
### loop over runs (30)


# data[0][0][0][0]  # data[setup][sample_size][run][train/test]






In [ ]:
n_runs = 30
test_size = 1000

In [ ]:
# create empty list to save all datasets
data = []
for i in range(n_setups):
    data.append([])

for i in range(n_setups):
    for s in range(4):
        data[i].append([])

for i in range(n_setups):
    for s in range(4):
        for r in range(n_runs):
            data[i][s].append([])

for i in range(n_setups):
    for s in range(4):
        for r in range(n_runs):
            for t in range(2):
                data[i][s][r].append([])

data

In [ ]:
# generate all datasets
s = 0
for setup in range(n_setups):
    for size in sample_sizes:
        for run in range(n_runs):
            for t in range(2):
                data[setup][s][run][t] = generate(ex=0.5, cate='glinear',sample_size=size)
        s += 1 # update index for sample_sizes
    s = 0 # reset index for sample_sizes

In [ ]:
data[0][3][0][0] # all the same

TODO: Change the generate function such that test size is always the same, you know?